In [ ]:
%matplotlib inline
import scipy.io as sio
from ML_models.POD_ESN import *

mat = sio.loadmat('data/circle_data.mat')
data = np.array([mat['ux'], mat['uy']])
data = np.einsum('u t x y -> u y x t',np.squeeze(data))

Nu, Ny, Nx, Nt = data.shape

N_test = int(.6 * Nt)
X_test = data[..., -N_test:].copy()
X_train = data[..., :-N_test].copy()

case = POD_ESN(X=X_train, dt=0.01, N_modes=5, rho_range = (.2, 1.05), Nq=5, 
               plot_decomposition=True, perform_test=False, plot_training=False)

case_og = case.copy()

In [ ]:


XX1, XX2 = case.domain_mesh()[:-1]

X1, X2 = XX1.ravel(), XX2.ravel()

plt.figure(figsize=(3,5))
plt.plot(XX1, XX2, marker='.', color='k', linestyle='none', ms=0.1)

X1_up, X2_up, grid_idx_up = case.domain_mesh(down_sample=[10, 20], ravel=True)

idx = grid_idx_up[(abs(X1_up) < 1.5) & (abs(X2_up-6) < 2)]

sensor_locations = case.sensor_locations[case.sensor_locations< len(X1)]

plt.plot(X1[idx], X2[idx], marker='o', color='b', linestyle='none', ms=1.5)
plt.plot(X1[sensor_locations], X2[sensor_locations], marker='o', color='r', linestyle='none', ms=5)


plt.xlabel('x1')
plt.ylabel('x2');

In [ ]:
# Plot training data at probes

ttt = np.arange(0, Nt-N_test) * case.dt
yyy = X_train.reshape((-1, Nt-N_test))[case.sensor_locations]
q_mean = case.Q_mean[case.sensor_locations]

qqq = case.get_observables(Phi=case.Phi)

fig, axs = plt.subplots(nrows=case.Nq//3, ncols=3, figsize=(10, 10), sharex=True, sharey='col', layout='constrained')

axs = axs.T.flatten()
axs[0].set(title="$u_x-\\bar{u}_x$")
axs[case.Nq//3].set(title="$u_y-\\bar{u}_y$")
axs[2*case.Nq//3].set(title="$p-\\bar{p}$")

for kk, ax in enumerate(axs):
    ax.axvspan(0, case.t_train/case.dt, facecolor='tab:green', alpha=0.125, zorder=-100, label='Train')
    ax.axvspan(case.t_train/case.dt, (case.t_train+case.t_val)/case.dt, facecolor='tab:blue', alpha=0.125, zorder=-100, label='Train')
    ax.plot(yyy[kk] - q_mean[kk], lw=4, color='k')
    ax.plot(qqq[kk] - q_mean[kk], lw=1., ls=(0, (10, 2)), color='red')

plt.suptitle('Training and validation data');


In [ ]:
Q = case.reconstruct(Phi=case.Phi[-1], Psi=case.Psi)

case.domain = [-2, 2, 12, 0]

X, Y = case.domain_mesh()[:-1]

ff, aa = plt.subplots(1, 3)
for zz, ax in zip(Q, aa):
    ax.contourf(X, Y, zz.squeeze().T, 100);

In [ ]:
Q = case.reconstruct(Phi=case.Phi[-1], Psi=case.Psi)
case.plot_reconstruct(original_data=X_train[...,-1], reconstructed_data=Q, dim=[0, 1]);

In [ ]:

case.plot_sensors(background_data=case.reconstruct(Q_mean=case.Q_mean*0., reshape=True), fluctuations = True);

In [ ]:
case = case_og.copy()

# Integrate the ESN in time
psi, t = case.time_integrate(Nt=Nt*5)
case.update_history(psi, t)

# Compare prediction and truth at probes
qqq = case.get_observable_hist(Nt*5)

yyy = X_test.reshape((-1, N_test))[case.sensor_locations]
q_mean = case.Q_mean[case.sensor_locations]




In [ ]:
fig, axs = plt.subplots(nrows=case.Nq//3, ncols=3, figsize=(10, 10), sharex=True, sharey='col', layout='constrained')

axs = axs.T.flatten()
axs[0].set(title="$u_x-\\bar{u}_x$")
axs[case.Nq//3].set(title="$u_y-\\bar{u}_y$")
axs[2*case.Nq//3].set(title="$p-\\bar{p}$")

for kk, ax in enumerate(axs):
    ax.plot(yyy[kk] - q_mean[kk], lw=4, color='dimgray')
    ax.plot(qqq[kk] - q_mean[kk], lw=1., ls=(0, (10, 1)), color='tab:orange')
    ax.set(xlim=(0, Nt*5))
    # ax.set(xlim=(0, N_test))

In [ ]:
Q = case.reconstruct(Phi=psi[-N_test])
case.plot_reconstruct(original_data=X_test[...,-N_test], reconstructed_data=Q, dim=[0, 1, 2]);
plt.show()

In [ ]:
XX = X_train.reshape((-1, Nt-N_test))
XX[np.isnan(XX)] = 0

W = np.dot(XX.T, XX) / (XX.shape[-1]-1)
SS = np.real(np.linalg.eig(W)[0])